# PID Control of an Inverted Pendulum with a Reaction Wheel
#### V. Hunter Adams (vha3@cornell.edu)

## Introduction

When the James Webb Space Telescope (or nearly any other modern spacecraft, for that matter) wants to change its orientation, how does it do so? It's not obvious how to turn something around in space where there's nothing (not even air) to push against. You might consider using thrusters, but those come at a cost! They require propellant. Propellant weighs a lot, and you can run out of it! A thruster shoots material into space when it fires, which can interfere with sensitive instrumentation. Furthermore, thruster fires tend to be impulsive in nature, which can excite vibrational modes in the spacecraft and don't allow for the ultra-precise control required to point at a distant star. Fortunately, thrusters aren't the only option for changing a spacecraft's orientation!

For changing the *orbit* of a spacecraft, you must have an external force (like a thruster) acting on the spacecraft. But to change the *orientation* of a spacecraft, we just need a torque! That torque *might* come from a thruster exerting a force, since $\bf{\tau} = \bf{F} \times \bf{r}$, but it can also come from a change in angular momentum, since $\tau = \frac{d \bf{L}}{dt}$. A spacecraft can change its orientation by *storing* angular momentum in an internal device, and then *exchanging* angular momentum between that device and the spacecraft body. Often, that device is a [reaction wheel](https://en.wikipedia.org/wiki/Reaction_wheel#James_Webb_Space_Telescope).

A reaction wheel is a heavy, spinning disc rigidly mounted inside the spacecraft. The reaction wheel + spacecraft is a closed system which must conserve angular momentum. So, if we make the reaction wheel spin slightly faster in the clockwise direction, the spacecraft body will compensate by spinning in the counterclockwise direction. Point a reaction wheel in each direction, and you've got 3-axis control! If you care to dive into the physics, please see [here](https://vanhunteradams.com/5160/ADCS/Supplements/ADCS.html).

In this lab, we will use a PID controller to modulate the speed of a reaction wheel at the end of an inverted pendulum. The torque that the reaction wheel exerts on the pendulum will balance it. In order for this to work, we must close the loop around a measurement of the pendulum angle. This angle will be estimated using a complementary filter of accelerometer and gyroscope measurements from an IMU.

<figure>
    <img align="center" width="300" height="500" src="Webb.png" alt='missing' />
    <center><figcaption>James Webb Space Telescope</figcaption></center>
</figure>

**Key concepts**: PID control, PWM, electrical isolation, complementary filters and sensor fusion, I2C, timers, timer interrupts, digital low-pass filtering, gyroscopes, accelerometers, UART, threading, VGA, fixed-point arithmetic

***

## Demonstration

Please see demo video below.

<iframe width="560" height="315" src="https://www.youtube.com/embed/1x1sSMSw6X4" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

***

## Reading

Experience shows that students prefer these webpages short. For that reason, please find the reading and background materials on the webpages linked below. *Please note that the information in these readings will be critical for completing the lab.*

### Engineering background
- [**Mechanical construction of the inverted pendulum**](./Pendulum_construction.html): Procedure for building the inverted pendulum.
- [**Data display**](./Digital_Lowpass_Filters.html): Describes the data that you must visualize in realtime, and some strategies for visualizing that data (digital lowpass filters).
- [**Motor circuit and PWM**](./HBridgeCircuit.html): This webpage describes the circuit that you will construct to safely drive the DC motor, and how to generate PWM.
- **Tuning the reaction wheel PID controller**
- [**VGA driver**](https://vanhunteradams.com/Pico/VGA/VGA.html): You will not be asked to implement a VGA driver, you will use the one described on this webpage.
- [**MPU-6050 datasheet**](https://invensense.tdk.com/wp-content/uploads/2015/02/MPU-6000-Datasheet1.pdf): The datasheet for the accelerometer/gyro that we'll be using.
- [**RP2040 datasheet**](https://datasheets.raspberrypi.com/rp2040/rp2040-datasheet.pdf): See sections on PWM and I2C.
- [**RP2040 C SDK**](https://datasheets.raspberrypi.com/pico/raspberry-pi-pico-c-sdk.pdf): See sections on PWM and I2C.


### Mathematical background
- [**Complementary filters**](./Complementary_Filters.html): A method for estimating angle from gyroscope and accelerometer measurements. This is a *great* algorithm for your toolbox. Quick, easy, and effective.
- [**Phenomenological introduction to PID controllers**](https://vanhunteradams.com/PID/PID.html): This webpage provides a phenomenological introduction to PID controllers, using a system that is different from this one.
- [**Analytical introduction to PID controllers**](https://vanhunteradams.com/PID/PID_Analytical.html): This webpage provides an analytical introduction to PID controllers, using a system that is different from this one.
- [**Reaction wheel attitude dynamics**](https://vanhunteradams.com/5160/ADCS/Supplements/ADCS.html): The physics of spacecraft attitude dynamics and determination. Not required reading, but of potential interest.

***

## Program organization

You may organize your program however you like. Here is a suggestion:

- **Protothreads** maintains the ISR-driven, millisecond-scale timing as part of the supplied system. Use this for all low-precision timing (can have several milliseconds jitter).
- **PID ISR** uses a timer or PWM interrupt to ensure an exact 1kHz control rate.
> - Clears the interrupt flag
> - Reads the MPU-6050 for accelerometer and gyro measurements
> - Uses these measurements in a complementary filter to compute arm angle.
> - Runs the PID control loop at 1000/sec using the angle measurements from the complementary filter and the desired angle
> - Sets a hardware PWM signals using output-compare units to control the motor using the command: `pwm_set_chan_level(slice_num, PWM_CHAN_X[A or B], pwm_on_time);`.
- **Main** sets up peripherals and protothreads then just schedules tasks, round-robin
- **Thread 1**
> - Takes user input from the serial interface to setup PID parameters and the desired angle.
- **Thread 2**
> - Diplays accelerometer angle, gyro angle, complementary angle, and low-passed control input on VGA display

***

## Weekly checkpoints

Note that these checkpoints are **cumulative**. In week 2, for example, you must have also completed all of the requirements from week 1.

#### Week one checkpoint

#### Week two checkpoint

#### Week three assignment

***

## Lab report

Your written lab report should include all the sections mentioned on the [policy page](https://vanhunteradams.com/Pico/CourseMaterials/Policy.html), and:

- A schematic of the circuit you built
- How you selected the PID gains
- An image of your VGA display (showing control input and estimated angle) during balancing
- An image of your VGA display (showing control input and estimated angle) after an impulse (i.e. after smacking it with a pencil).
- A heavily commented listing of your code.

***